# Resolução de sistemas de equações lineares

## Importações

In [2]:
import numpy as np

| 0 + x + 3x + 2x + 4x = 3 
| 8x - 2x + 9x -x + 2x = -5 
| 5x + x + x + 7x + 2x = 6
| -2x + 4x + 5x + x + 0 = -1
| 7x - 3x + 2x - 4x + x = 8

## Definições

In [3]:
matrix1 = np.array([
          [0, 1, 3, 2, 4, 3],
          [8, 2, 9, -1, 2, -5],
          [5, 1, 1, 7, 2, 6],
          [-2, 4, 5, 1, 0, -1],
          [7, -3, 2, -4, 1, 8]
          ], dtype='float')  

#### Eliminação de Gauss s/ Pivotamento

In [7]:
def gauss_elimination(A, B):

  if A[0, 0] == 0:
    print('Não é possível resolver o sistema, pivô nulo!')
    return None

  # superior matrix
  for base in range(len(A) - 1):
    for row in range(base + 1, len(A)):
      factor = A[row, base] / A[base, base]
      A[row, base] = 0

      for col in range(base + 1, len(A)):
        A[row, col] = A[row, col] - factor * A[base, col]

      B[row] = B[row] - factor * B[base]

  return A, B


In [8]:
gauss_elimination(matrix1, B2)

Não é possível resolver o sistema, pivô nulo!
